In [2]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import codecs

url = 'http://web.mta.info/developers/turnstile.html'

response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

soup.findAll('a')

In [6]:
core_url = 'https://www.gutenberg.org/'
titles_url = 'https://www.gutenberg.org/browse/titles/'
alphabet = 'abcdefghijklmnopqrstuvwxyz'
gb_urls = [titles_url + letter for letter in alphabet]

books_list = []
for url in gb_urls[:1]:
    books_list.extend(get_book_list(url))

print('Number of books found: ' + str(len(books_list)))
complete_text = ''
for book_url in books_list[:1000]:
    complete_text += get_book_text(book_url)

f = codecs.open('test_file.txt', 'w', 'utf-8')
f.write(complete_text)
f.close()
    
print('Success')

Number of books found: 8068


KeyboardInterrupt: 

In [4]:
def get_book_list(input_url):
    core_url = 'https://www.gutenberg.org/'
    
    response = requests.get(input_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    books_a = soup.find_all('h2')
    better_books_a = [book for book in books_a if '(English)' in book.get_text()]
    book_ids_a = [book.find_all('a')[0]['href'].split('/')[-1] for book in better_books_a]
    urls_a = [core_url + 'cache/epub/' + book_id + '/pg' + book_id + '.txt' for book_id in book_ids_a]
    urls_b = [core_url + 'files/' + book_id + '/' + book_id + '-0.txt' for book_id in book_ids_a]
    urls_a.extend(urls_b)
    
    return urls_a

In [5]:
def get_book_text(input_url):
    #print(input_url)
    response = requests.get(input_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    #print(soup)
    raw_text = soup.find_all(text=True)
    if len(raw_text) == 1:
        first_split = raw_text[0].split('*** START OF THIS PROJECT GUTENBERG EBOOK')
        if len(first_split) == 2:
            second_split = first_split[1].split('*** END OF THIS PROJECT GUTENBERG EBOOK')[0]
        else:
            re_first_split = first_split[0].split('*** START OF THE PROJECT GUTENBERG EBOOK')
            if len(re_first_split) == 2:
                second_split = re_first_split[1].split('*** END OF THE PROJECT GUTENBERG EBOOK')[0]
            else:
                second_split = ''
        text = second_split
    else:
        text = ''
    
    return text